## LSTM class Developed
-----------------------------
[feat] 7/16
- ! to prevent overfitting use K-fold, validation split ! <- NEED TO DEVELOP
- basical LSTM model => lstm dense need to doubled
- not tested
- without AOC, interest rate
- bidirectional LSTM Model

[feat] 7/17

[need] 7/17
- val_loss 범위 설정
- k-fold 구현
- return값 설정
- 모델 예측 설정 방법







imorts & get datas
------------

In [3]:
#!pip install -U finance-datareader

#%pip install FinanceDataReader

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
#import FinanceDataReader as fdr

In [5]:
data=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/KRX_DATA/train.csv')

In [6]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목명",inplace=False)
subject_Data.head()
subject_Data.columns

pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])

divided = pv_s_data.index.get_level_values('종목코드').unique()

sliced_dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    sliced_dataframes.append(sliced_dataframe)

In [7]:
data=sliced_dataframes

In [8]:
data[0].shape

(494, 6)

## LSTM Obj

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
#import FinanceDataReader as fdr

class LSTM_Object():

  def __init__(self,data):
    #get sliced data
    self.data=data

  def modeling(self):

    self.prescale()
    self.model=Sequential()
    #layers
    self.model.add(Bidirectional(LSTM(1024,return_sequences=True,input_shape=(self.time_steps, self.num_features))))
    #self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #self.model.add(Dropout(0.25))
    #self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #self.model.add(Dropout(0.25))
    self.model.add(Dense(512, activation='relu'))
    self.model.add(Dense(256, activation='relu'))
    self.model.add(Dense(128, activation='relu'))
    #self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #output layer
    self.model.add(Dense(1, activation='relu'))

    self.filename = join('/content/drive/MyDrive/2023_1st_vac/KRX_modelings/tests', 'ckeckpointer.ckpt')
    checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True,monitor='val_loss',verbose=1)

    earlystopping = EarlyStopping(monitor='val_loss', patience=1000)
    self.model.compile(loss = 'mean_squared_error', optimizer=Adam(learning_rate=0.001))
    self.history=self.model.fit(self.x_train,self.y_train,epochs=1,batch_size=128,validation_data=(self.x_val,self.x_val),shuffle=False,callbacks=[checkpoint, earlystopping])


  def prescale(self):
    x_scaler = MinMaxScaler()
    self.y_scaler = MinMaxScaler()
    # Scaling features
    x_data = self.data[['거래량', '고가', '시가', '저가']]
    x_data = x_scaler.fit_transform(x_data)
    # Scaling target variable (종가)
    y_data = self.data['종가']
    y_data = self.y_scaler.fit_transform(y_data.values.reshape(-1, 1))
    # validation set
    x_train_p, x_test, y_train_p, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)
    x_train, x_val, y_train, y_val = train_test_split(x_train_p, y_train_p, test_size=0.25, shuffle=False)
    ### reshape input datas()

    train_size=len(x_train)
    val_size=len(x_val)
    test_size=len(x_test)

    self.num_features = x_data.shape[1]
    self.time_steps=30

    self.x_train = self.create_sequences(x_train)
    self.x_test = self.create_sequences(x_test)
    self.x_val = self.create_sequences(x_val)

    self.y_train=y_train
    self.y_test=y_test
    self.y_val=y_val


  def create_sequences(self, data):
    num_samples, num_features = data.shape
    sequences = []
    for i in range(num_samples - self.time_steps +1):
      sequences.append(data[i:i + self.time_steps, :])
    return np.array(sequences)

  def ret_learning(self):
    # rescale to get real value

    self.prescale()
    self.modeling()
    self.model.load_weights(self.filename)
    pred = self.model.predict(self.x_test)

    #rescaled_pred = self.y_scaler.inverse_transform(pred)
    #rescaled_real = self.y_scaler.inverse_transform(self.y_test)
    #real_prices = []
    #pred_prices = []




    print(pred)



    #plt.figure(figsize=(12, 9))
    #plt.plot(np.asarray(rescaled_real), label='actual')
    #plt.plot(pred, label='prediction')
    #plt.legend()
    #plt.show()

In [26]:
test_1=data[0]

In [27]:
L=LSTM_Object(test_1)
L.prescale()

In [28]:
L.x_train.info()


AttributeError: ignored

In [29]:
L.y_train


array([[0.54124579],
       [0.53282828],
       [0.54124579],
       [0.5496633 ],
       [0.58754209],
       [0.57912458],
       [0.5496633 ],
       [0.58333333],
       [0.56649832],
       [0.54545455],
       [0.54124579],
       [0.53703704],
       [0.52861953],
       [0.52861953],
       [0.53282828],
       [0.52861953],
       [0.53703704],
       [0.60858586],
       [0.60858586],
       [0.63804714],
       [0.62962963],
       [0.62962963],
       [0.64225589],
       [0.63804714],
       [0.65909091],
       [0.63383838],
       [0.66750842],
       [0.6043771 ],
       [0.57070707],
       [0.57912458],
       [0.56228956],
       [0.55387205],
       [0.56228956],
       [0.55808081],
       [0.58333333],
       [0.56649832],
       [0.55808081],
       [0.55387205],
       [0.54124579],
       [0.52441077],
       [0.52441077],
       [0.52861953],
       [0.54124579],
       [0.54545455],
       [0.54545455],
       [0.60016835],
       [0.59175084],
       [0.583

In [30]:
L.ret_learning()

ValueError: ignored